# Part 1 - Working with Text Data

### Use Python string methods remove irregular whitespace from the following string:

In [3]:
# All imports in one place whether I need them or not. 
import pandas as pd
import numpy as np
import re
import requests
import string
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

from collections import defaultdict
from collections import Counter

!pip install -U nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.4)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Removing the unnessary white space to make a normal sentence
whitespace_string = '\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   '
print(whitespace_string.strip())
print('---------------')
print(' '.join(whitespace_string.split()))

This is a    string   that has  
 a lot of  extra 
   whitespace.
---------------
This is a string that has a lot of extra whitespace.


### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://github.com/ryanleeallred/datasets/blob/master/dates.txt)

- Day
- Month
- Year


In [5]:
# Bringing in the text file to use and make a page with the required information
url = ('https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt')
page= requests.get(url)
page = page.text
page

'March 8, 2015\r\nMarch 15, 2015\r\nMarch 22, 2015\r\nMarch 29, 2015\r\nApril 5, 2015\r\nApril 12, 2015\r\nApril 19, 2015\r\nApril 26, 2015\r\nMay 3, 2015\r\nMay 10, 2015\r\nMay 17, 2015\r\nMay 24, 2015\r\nMay 31, 2015\r\nJune 7, 2015\r\nJune 14, 2015\r\nJune 21, 2015\r\nJune 28, 2015\r\nJuly 5, 2015\r\nJuly 12, 2015\r\nJuly 19, 2015'

In [6]:
# Separate the month, day, and year
month = re.findall(r"[a-zA-z]+", page)
year = re.findall(r"\d{4}", page)
day = re.findall(r"\d+",page)
day = day[::2]
# Create the dataframe
date = pd.DataFrame({'Month ':month,  'Day ': day,  'Year ':year})
date.head()

,Day,Month,Year
0,8,March,2015
1,15,March,2015
2,22,March,2015
3,29,March,2015
4,5,April,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [7]:
# Bringing in the data to read
url = 'https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv'
# Read the data
twitters = pd.read_csv(url)
twitters.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [8]:
twitters.isnull().sum()

Sentiment        0
SentimentText    0
dtype: int64

In [18]:
# Making sentiment data readable
twitters['SentimentText'] = twitters['SentimentText'].apply(lambda x: re.sub(r'[^\w]', ' ', x))
twitters.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7 30 O
3,0,Omgaga Im sooo im gunna CRy I ...
4,0,i think mi bf is cheating on me ...


In [23]:
# Do everything at once. re: from Ryan's notes
table = str.maketrans('', '', string.punctuation)
stop_words = set(stopwords.words('english'))

def nltk_tokenize(input):

    # word
    tokens = word_tokenize(input)
    #print("Tokens:", tokens)
    #lowercase
    lowercase_tokens = [w.lower() for w in tokens]
    #print("Lowercase:", lowercase_tokens)
    # Strip punctuation
    no_punctuation = [x.translate(table) for x in lowercase_tokens]
    # Remove words that aren't alphabetic
    alphabetic = [word for word in no_punctuation if word.isalpha()]
    # Remove stopwords
    words = [w for w in alphabetic if not w in stop_words]
    return words
  

# Create a new column with the tokenized data
twitters['tokenized'] = twitters['SentimentText'].apply(nltk_tokenize)
twitters.head()

,Sentiment,SentimentText,tokenized
0,0,is so sad for my APL frie...,"[sad, apl, friend]"
1,0,I missed the New Moon trail...,"[missed, new, moon, trailer]"
2,1,omg its already 7 30 O,"[omg, already]"
3,0,Omgaga Im sooo im gunna CRy I ...,"[omgaga, im, sooo, im, gunna, cry, dentist, si..."
4,0,i think mi bf is cheating on me ...,"[think, mi, bf, cheating, tt]"


### How should TF-IDF scores be interpreted? How are they calculated?

#### Your Answer Here #####

# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



In [0]:
##### Your Code Here #####

# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [0]:
##### Your Code Here #####